In [0]:
%pip install --upgrade openpyxl

In [0]:
%run ./databricks_data_objects_setup

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


DataFrame[]

DataFrame[]

DataFrame[]

In [0]:
%run ./sales_data_processing_helper

DataFrame[]

DataFrame[]

path,name,size,modificationTime
dbfs:/Volumes/sales/bronze/raw_data_files/Customer.xlsx,Customer.xlsx,89424,1754124106000
dbfs:/Volumes/sales/bronze/raw_data_files/Orders.json,Orders.json,3045599,1754124106000
dbfs:/Volumes/sales/bronze/raw_data_files/Products.csv,Products.csv,173396,1754124106000


In [0]:
customer_data_file_path = "/Volumes/sales/bronze/raw_data_files/Customer.xlsx"
orders_data_file_path = "/Volumes/sales/bronze/raw_data_files/Orders.json"
products_data_file_path = "/Volumes/sales/bronze/raw_data_files/Products.csv"

catalog_name = "sales"

raw_database_name = "bronze"

enriched_database_name = "silver"

aggregated_database_name = "gold"

raw_customers_table_name = f"{catalog_name}.{raw_database_name}.customers_raw"
raw_orders_table_name = f"{catalog_name}.{raw_database_name}.orders_raw"
raw_products_table_name = f"{catalog_name}.{raw_database_name}.products_raw"

cleaned_customers_table_name = f"{catalog_name}.{enriched_database_name}.customers_cleaned"
cleaned_orders_table_name = f"{catalog_name}.{enriched_database_name}.orders_cleaned"
cleaned_products_table_name = f"{catalog_name}.{enriched_database_name}.products_cleaned"

enriched_orders_table_name = f"{catalog_name}.{enriched_database_name}.sales_enriched_master"

aggregated_profit_table_name = f"{catalog_name}.{aggregated_database_name}.aggregated_profit"

In [0]:
# Task 1
raw_customers_df = read_raw_customers(customer_data_file_path)
write_df_to_table(raw_customers_df, raw_customers_table_name)
raw_orders_df = read_raw_orders(orders_data_file_path)
write_df_to_table(raw_orders_df, raw_orders_table_name)
raw_products_df = read_raw_products(products_data_file_path)
write_df_to_table(raw_products_df, raw_products_table_name)

# Task 2
cleaned_customers_df = clean_customers(raw_customers_df)
write_df_to_table(cleaned_customers_df, cleaned_customers_table_name)
cleaned_orders_df = clean_orders(raw_orders_df)
write_df_to_table(cleaned_orders_df, cleaned_orders_table_name)
cleaned_products_df = clean_products(raw_products_df)
write_df_to_table(cleaned_products_df, cleaned_products_table_name)

# Task 3
enriched_orders_master_df = create_enriched_orders_df(cleaned_customers_df, cleaned_orders_df, cleaned_products_df)
write_df_to_table(enriched_orders_master_df, enriched_orders_table_name)

# Task 4
aggregated_profit_df = create_aggregated_profit_df(enriched_orders_master_df)
write_df_to_table(aggregated_profit_df, aggregated_profit_table_name)

INFO:__main__:Table sales.bronze.customers_raw created successfully
INFO:__main__:Table sales.bronze.orders_raw created successfully
INFO:__main__:Table sales.bronze.products_raw created successfully
INFO:__main__:Table sales.silver.customers_cleaned created successfully
INFO:__main__:Table sales.silver.orders_cleaned created successfully
INFO:__main__:Table sales.silver.products_cleaned created successfully
INFO:__main__:Table sales.silver.sales_enriched_master created successfully
INFO:__main__:Table sales.gold.aggregated_profit created successfully


In [0]:
# Task 5
# a) Profit by Year
spark.sql("""
SELECT year, SUM(total_profit) AS total_profit
FROM sales.gold.aggregated_profit
GROUP BY year
ORDER BY year
""").display()

year,total_profit
2014,10248.02
2015,5875.57
2016,-8507.62
2017,17492.90


In [0]:
# Task 5
# b) Profit by Year + Product Category
spark.sql("""
SELECT year, category, SUM(total_profit) AS total_profit
FROM sales.gold.aggregated_profit
GROUP BY year, category
ORDER BY year, category
""").display()

year,category,total_profit
2014,Furniture,1483.06
2014,Office Supplies,3287.37
2014,Technology,5477.59
2015,Furniture,-1902.84
2015,Office Supplies,791.91
2015,Technology,6986.50
2016,Furniture,13.69
2016,Office Supplies,6016.33
2016,Technology,-14537.64
2017,Furniture,323.51


In [0]:
# Task 5
# c) Profit by Customer
spark.sql("""
SELECT customer_id, customer_name, SUM(total_profit) AS total_profit
FROM sales.gold.aggregated_profit
GROUP BY customer_id, customer_name
ORDER BY total_profit DESC
""").display()

customer_id,customer_name,total_profit
PR-18880,PatrickRyan,5432.44
NM-18445,NathanMautz,2076.01
SR-20740,StevenRoelle,1695.07
MC-17425,MarkCousins,1510.86
PK-19075,PeteKriz,1505.86
AB-10060,AdamBellavance,1445.30
KW-16435,KatrinaWillman,1435.19
DM-13015,DarrinMartin,1397.47
DM-13345,DeniseMonton,1267.60
ME-17320,MariaEtezadi,1217.92


In [0]:
# Task 5
# d) Profit by Customer + Year
spark.sql("""
SELECT year, customer_id, customer_name, SUM(total_profit) AS total_profit
FROM sales.gold.aggregated_profit
GROUP BY year, customer_id, customer_name
ORDER BY year, total_profit DESC
""").display()

year,customer_id,customer_name,total_profit
2014,NM-18445,NathanMautz,2076.01
2014,DM-13015,DarrinMartin,1397.47
2014,ME-17320,MariaEtezadi,1020.49
2014,YC-21895,YosephCarroll,807.20
2014,GT-14710,GregTran,676.46
2014,LR-16915,LenaRadford,490.73
2014,AM-10705,AnneMcFarland,481.47
2014,LD-17005,LisaDeCherney,362.84
2014,VD-21670,ValerieDominguez,305.13
2014,RA-19885,RubenAusman,290.30
